In [255]:
import numpy as np
import pandas as pd
import sklearn.linear_model

In [256]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str,
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int,
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [257]:
def numpy_data(data_pd, features, output):
    data_pd['constant'] = 1
    features = ['constant'] + features
    feature_matrix = np.array(data_pd[features], dtype = np.float32)
    output_array = np.array(data_pd[output])
    
    return (feature_matrix, output_array)

In [258]:
def predict_outcome(feature_matrix, weights):
    predictions = feature_matrix.dot(weights)
    return(predictions)

In [259]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis = 0)
    X_normalized = features / norms
    
    return (X_normalized, norms)

In [260]:
data = pd.read_csv("kc_house_data.csv", dtype= dtype_dict)

features1 = ['sqft_living', 'bedrooms']
feature_matrix, output = numpy_data(data, features1, 'price')

In [261]:
feature_matrix_norm, norms = normalize_features(feature_matrix)

In [262]:
initial_weights = [1., 4., 1.]

In [263]:
prediction = predict_outcome(feature_matrix_norm, initial_weights)

In [264]:
residual = np.subtract(data['price'], prediction)
ro = [0.] * 3
for i in xrange(len(initial_weights)):
    ff = np.multiply(initial_weights[i], feature_matrix_norm[:,i])
    ff = residual + ff
    ro[i] = np.dot(ff, feature_matrix_norm[:,i])


In [265]:
print ro

[79400297.894738987, 87939474.579189181, 80966701.158269122]


##Quiz Question: Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

In [266]:
print "For feature1 l1_penalty={0}".format(ro[1] * 2)
print "For feature2 l1_penalty={0}".format(ro[2] * 2)

For feature1 l1_penalty=175878949.158
For feature2 l1_penalty=161933402.317


#Quiz Question: What range of values of l1_penalty would set both w[1] and w[2] to zero, if we were to take a step in that coordinate?

In [267]:
print "Value of l1_penalty greather than {0}".format(max(ro[1]*2, ro[2]*2))

Value of l1_penalty greather than 175878949.158


## Single Coordinate Descent Step

In [268]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    prediction = predict_outcome(feature_matrix, weights)
    ro_i = np.sum(feature_matrix[:,i]*(output - prediction + weights[i]*feature_matrix[:,i]))
    if i == 0: 
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

#### Testing lasso coordinate descent step

In [269]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent

In [270]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    max_step = float('inf')
    n = len(initial_weights)
    weights = initial_weights.copy()
    while max_step > tolerance:
        new_weights = weights.copy()
        for i in xrange(n):
            new_weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, new_weights, l1_penalty)
        max_step = max([abs(x - y) for x,y in zip(new_weights, weights)])
        print(max_step)
        weights = new_weights
        
    return weights

In [271]:
initial_weights = np.zeros(3)

In [272]:
l1_penalty = 1e7

In [273]:
tolerance = 1.0

In [274]:
data = pd.read_csv("kc_house_data.csv", dtype= dtype_dict)

features1 = ['sqft_living', 'bedrooms']
feature_matrix, output = numpy_data(data, features1, 'price')
feature_matrix_norm, norms = normalize_features(feature_matrix)

simple_weights = lasso_cyclical_coordinate_descent(feature_matrix_norm, output, initial_weights, l1_penalty, tolerance)

79400302.5179
9138170.01931
8194810.94739
6598908.64094
5522176.82669
4621131.5355
3867111.4047
3236119.12337
2708088.36084
2266218.58703
1896440.03312
1587003.16224
1328054.21346
1111355.40081
930021.007128
778267.576597
651281.660543
545012.899507
456084.3685
381665.16789
319387.99302
267275.059868
223665.488
187169.540772
156631.525265
131073.88755
109687.219926
91790.0986186
76812.5857342
64278.2696454
53786.9875125
45013.3541733
37666.0611886
31522.1222176
26376.7134891
22076.4151101
18475.1005221
15455.6456428
12937.6384464
10825.0136782
9061.88481971
7582.71190004
6344.61554163
5307.22139587
4442.49342977
3718.72162504
3109.07869013
2601.99050955
2179.30151131
1822.40869008
1524.70904145
1276.87536058
1069.47558172
892.932380516
746.530092664
625.330875918
523.074683949
438.058347564
366.706711549
311.518038992
255.617589597
216.186915666
178.305168569
148.187933087
128.153222177
105.066576809
87.6841048114
73.8504106812
60.3295640461
51.3874487653
43.6659588963
38.2801195905
31

In [275]:
print simple_weights

[ 21624964.63404675  63157280.82672167         0.        ]


#Quiz Question: What is the RSS of the learned model on the normalized dataset?

In [276]:
predictions = predict_outcome(feature_matrix_norm, simple_weights)
rss = sum([(x-y)**2 for x,y in zip(predictions, output)])
print rss

1.63049211502e+15


#Quiz Question: Which features had weight zero at convergence?

In [277]:
names = ['intercept'] + features1
print zip(names, simple_weights)

[('intercept', 21624964.634046748), ('sqft_living', 63157280.826721668), ('bedrooms', 0.0)]


###Evaluating LASSO fit with more features

In [278]:
train = pd.read_csv('kc_house_train_data.csv', dtype = dtype_dict)
test  = pd.read_csv('kc_house_test_data.csv' , dtype = dtype_dict)

In [279]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront',
            'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

In [280]:
feature_matrix_train, output_train = numpy_data(train, features, 'price')
feature_matrix_test, output_test = numpy_data(test, features, 'price')

In [281]:
feature_matrix_train_norm, norms_train = normalize_features(feature_matrix_train)

In [282]:
l1_penalty = 1e7
tolerance = 1
initial_weights = np.zeros(1+len(features))
weights1e7 = lasso_cyclical_coordinate_descent(feature_matrix_train_norm, output_train, initial_weights, l1_penalty, tolerance)
full_feature1 = ['intercept'] + features

71114622.3979
5024364.16646
5012327.52386
5000679.40789
4991246.44015
4984366.95591
4979511.17881
4973807.71438
2879225.43028
2706153.51763
2532296.79006
2368321.10773
2213127.75238
1552454.85339
507838.582463
438812.961151
371481.120212
313412.285091
264244.076348
222755.168268
187769.537601
158280.758895
133419.929856
112467.308541
94801.6943416
79914.0418673
67359.9356365
56780.6073409
47863.3772326
40347.8853403
34008.6370201
28669.8691455
24167.8293467
20370.1453181
17170.7302933
14474.2714256
12201.5055174
10283.6587528
8669.40460893
7307.1193908
6159.99084576
5191.22608656
4376.46224042
3691.08480185
3111.73002204
2622.70618035
2208.07640421
1862.50122612
1569.76806283
1321.79417718
1117.82492193
943.156350952
792.697701298
669.155771405
559.915749256
472.778069925
400.014451802
338.603125602
284.689951107
237.000023432
202.21415573
170.239466257
145.533553362
120.051348649
102.852841582
83.7823702991
71.6311934255
61.2407078482
52.3738193139
45.0778751262
35.6551993601
30.18423

##  Quiz Question: What features had non-zero weight in this case?

In [283]:
print zip(full_feature1, weights1e7)

[('intercept', 24429565.23562859), ('bedrooms', 0.0), ('bathrooms', 0.0), ('sqft_living', 48389212.057468221), ('sqft_lot', 0.0), ('floors', 0.0), ('waterfront', 3317511.7871658709), ('view', 7329956.6112941541), ('condition', 0.0), ('grade', 0.0), ('sqft_above', 0.0), ('sqft_basement', 0.0), ('yr_built', 0.0), ('yr_renovated', 0.0)]


In [293]:
l1_penalty = 1e8
tolerance = 2.
weights1e8 = lasso_cyclical_coordinate_descent(feature_matrix_train_norm, output_train, initial_weights, l1_penalty, tolerance)

71114622.3979
1.9932692349


# Quiz Question: What features had non-zero weight in this case?

In [294]:
print zip(full_feature1, weights1e8)

[('intercept', 71114620.404586002), ('bedrooms', 0.0), ('bathrooms', 0.0), ('sqft_living', 0.0), ('sqft_lot', 0.0), ('floors', 0.0), ('waterfront', 0.0), ('view', 0.0), ('condition', 0.0), ('grade', 0.0), ('sqft_above', 0.0), ('sqft_basement', 0.0), ('yr_built', 0.0), ('yr_renovated', 0.0)]


In [285]:
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(feature_matrix_train_norm, output_train, initial_weights, l1_penalty, tolerance)

71114622.3979
8441277.54274
6789874.48413
5538915.37507
4527799.68887
3700363.95256
3022495.39217
2468072.13926
2015787.95289
1647983.91106
1389757.21243
1334789.73554
1293150.57179
1261720.69464
1238003.84478
1220010.49033
1206165.77815
1194976.2662
1185739.4927
1177724.12085
1170368.49609
1163252.35645
1156069.8513
1148606.54012
1140711.26236
1132298.86037
1123318.56394
1113751.48713
1103608.57288
1092910.26466
1081693.45585
1070012.18886
1057906.15521
1045430.26512
1032645.54058
1019595.98642
1006339.90768
992920.649906
979388.103387
965787.624407
952152.665576
938523.163907
924935.398069
911417.121883
897989.528843
884687.03573
871516.922055
858513.25499
845678.902182
832358.210163
819806.342156
807540.816234
795526.102892
783745.023824
772192.982331
760875.52329
749790.391892
738949.630978
728142.607008
717345.583339
706816.221229
696574.618176
686519.017256
676760.167672
667281.597809
658070.929505
649118.491155
640413.25681
631938.760247
623703.486915
615683.802673
607873.889356

## Quiz Question: What features had non-zero weight in this case?

In [286]:
print zip(full_feature1, weights1e4)

[('intercept', 78564820.631234437), ('bedrooms', -22097301.914200727), ('bathrooms', 12791113.371316463), ('sqft_living', 93808526.078915238), ('sqft_lot', -2013174.9612432055), ('floors', -4219204.9888766222), ('waterfront', 6482843.4415590316), ('view', 7127405.4416476879), ('condition', 5001684.7571378797), ('grade', 14327471.498521565), ('sqft_above', -15771338.156859603), ('sqft_basement', -5159703.2583063636), ('yr_built', -84495509.249537185), ('yr_renovated', 2824440.8464404163)]


# Rescaling learned weights

In [287]:
normalized_weights1e4 = weights1e4 / norms_train
normalized_weights1e7 = weights1e7 / norms_train
normalized_weights1e8 = weights1e8 / norms_train
print normalized_weights1e7[3]

161.317588635


## Evaluating all models on test data

In [289]:
predictions_weights1e4 = predict_outcome(feature_matrix_test, normalized_weights1e4)
predictions_weights1e7 = predict_outcome(feature_matrix_test, normalized_weights1e7)
predictions_weights1e8 = predict_outcome(feature_matrix_test, normalized_weights1e8)

In [291]:
rss_weights1e4 = sum([(x-y)**2 for x,y in zip(output_test, predictions_weights1e4)])
rss_weights1e7 = sum([(x-y)**2 for x,y in zip(output_test, predictions_weights1e7)])
rss_weights1e8 = sum([(x-y)**2 for x,y in zip(output_test, predictions_weights1e8)])

In [292]:
print rss_weights1e4
print rss_weights1e7
print rss_weights1e8

2.28459948517e+14
2.75962004975e+14
6.15751251045e+14
